### Imports and Load Dataset

In [1]:
import numpy as np
import pandas as pd

import matplotlib as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

2024-01-05 20:16:27.857630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 20:16:27.857667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 20:16:27.858774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 20:16:27.867164: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 20:16:28.649240: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv('hotel.csv')

In [3]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
df.size

3820480

In [5]:
df.shape

(119390, 32)

### EDA

In [6]:
df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [7]:
df.isnull().mean()*100

hotel                              0.000000
is_canceled                        0.000000
lead_time                          0.000000
arrival_date_year                  0.000000
arrival_date_month                 0.000000
arrival_date_week_number           0.000000
arrival_date_day_of_month          0.000000
stays_in_weekend_nights            0.000000
stays_in_week_nights               0.000000
adults                             0.000000
children                           0.003350
babies                             0.000000
meal                               0.000000
country                            0.408744
market_segment                     0.000000
distribution_channel               0.000000
is_repeated_guest                  0.000000
previous_cancellations             0.000000
previous_bookings_not_canceled     0.000000
reserved_room_type                 0.000000
assigned_room_type                 0.000000
booking_changes                    0.000000
deposit_type                    

In [8]:
df.drop(columns=['company'], inplace=True)

In [9]:
df.dropna(axis=0, inplace=True)

In [10]:
df.shape

(102894, 31)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102894 entries, 3 to 119389
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           102894 non-null  object 
 1   is_canceled                     102894 non-null  int64  
 2   lead_time                       102894 non-null  int64  
 3   arrival_date_year               102894 non-null  int64  
 4   arrival_date_month              102894 non-null  object 
 5   arrival_date_week_number        102894 non-null  int64  
 6   arrival_date_day_of_month       102894 non-null  int64  
 7   stays_in_weekend_nights         102894 non-null  int64  
 8   stays_in_week_nights            102894 non-null  int64  
 9   adults                          102894 non-null  int64  
 10  children                        102894 non-null  float64
 11  babies                          102894 non-null  int64  
 12  meal                 

In [12]:
df.hotel.value_counts()

hotel
City Hotel      71181
Resort Hotel    31713
Name: count, dtype: int64

In [13]:
df['hotel'] = df['hotel'].apply(lambda x: 1 if x == 'Resort Hotel' else 0)

In [14]:
df['arrival_date_month'].replace({'August':8, 'July': 7, 'May': 5, 'October': 10, 'April': 4, 'June': 6,
                                 'September': 9, 'March': 3, 'February': 2, 'November': 11, 
                                 'December': 12, 'January': 1}, inplace=True)

In [15]:
df['arrival_date_month'].value_counts()

arrival_date_month
8     12672
7     11599
5     10087
6      9803
4      9746
10     9437
9      9411
3      8132
2      6445
12     5620
11     5396
1      4546
Name: count, dtype: int64

In [16]:
df.meal.value_counts()

meal
BB           77638
HB           13421
SC           10334
Undefined      830
FB             671
Name: count, dtype: int64

In [17]:
df.country.value_counts()

country
PRT    37591
GBR    10889
FRA     9810
ESP     7848
DEU     6998
       ...  
BHS        1
UMI        1
NPL        1
MWI        1
BFA        1
Name: count, Length: 174, dtype: int64

In [18]:
df.market_segment.value_counts()

market_segment
Online TA        56057
Offline TA/TO    23774
Groups           15661
Direct            6582
Corporate          692
Complementary      103
Aviation            25
Name: count, dtype: int64

In [19]:
df.distribution_channel.value_counts()

distribution_channel
TA/TO        94575
Direct        7003
Corporate     1125
GDS            191
Name: count, dtype: int64

In [20]:
df.reserved_room_type.value_counts()

reserved_room_type
A    73265
D    17535
E     5457
F     2537
G     1809
B      992
C      765
H      534
Name: count, dtype: int64

In [21]:
df.assigned_room_type.value_counts()

assigned_room_type
A    64502
D    21938
E     6306
F     3134
G     2169
B     1879
C     1868
H      601
I      274
K      223
Name: count, dtype: int64

In [22]:
df.deposit_type.value_counts()

deposit_type
No Deposit    90324
Non Refund    12522
Refundable       48
Name: count, dtype: int64

In [23]:
df['deposit_type'] = df['deposit_type'].replace({'Refundable':2, 'Non Refund':1, 'No Deposit':0})

In [24]:
df.customer_type.value_counts()

customer_type
Transient          77756
Transient-Party    20602
Contract            4063
Group                473
Name: count, dtype: int64

In [25]:
df['customer_type'] = df['customer_type'].replace({'Transient':0, 'Transient-Party':1, 'Group':2, 'Contract':3})

In [26]:
df['reservation_status'].value_counts()

reservation_status
Check-Out    62733
Canceled     39158
No-Show       1003
Name: count, dtype: int64

In [27]:
df['reservation_status'] = df['reservation_status'].replace({'No-Show':0, 'Canceled':0, 'Check-Out':1})

In [28]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [29]:
df['year'] = df['reservation_status_date'].dt.year

In [30]:
df['month'] = df['reservation_status_date'].dt.month

In [31]:
df['day'] = df['reservation_status_date'].dt.day

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102894 entries, 3 to 119389
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           102894 non-null  int64         
 1   is_canceled                     102894 non-null  int64         
 2   lead_time                       102894 non-null  int64         
 3   arrival_date_year               102894 non-null  int64         
 4   arrival_date_month              102894 non-null  int64         
 5   arrival_date_week_number        102894 non-null  int64         
 6   arrival_date_day_of_month       102894 non-null  int64         
 7   stays_in_weekend_nights         102894 non-null  int64         
 8   stays_in_week_nights            102894 non-null  int64         
 9   adults                          102894 non-null  int64         
 10  children                        102894 non-null  float64     

In [33]:
df.drop(columns=['year', 'reservation_status_date'], inplace=True)

In [34]:
df = pd.get_dummies(df, columns=['meal', 'country', 'market_segment', 'distribution_channel', 
                                 'reserved_room_type', 'assigned_room_type'], dtype=int)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102894 entries, 3 to 119389
Columns: 234 entries, hotel to assigned_room_type_K
dtypes: float64(3), int32(2), int64(229)
memory usage: 183.7 MB


### Split 

In [36]:
X = df.drop(columns='is_canceled')
y = df['is_canceled']

In [37]:
X_train = X.sample(frac=.8, random_state=0)
y_train = y.sample(frac=.8, random_state=0)
X_test = X.drop(X_train.index)
y_test = y.drop(y_train.index)

### Construct the Layer Class

In [38]:
class SimpleLayer(keras.layers.Layer):
    
    def __init__(self, units=32, activation=None):
        super(SimpleLayer, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)
        super().build(input_shape)


    def call(self, inputs):
        
        # pass the computation to the activation layer
        return self.activation(tf.matmul(inputs, self.w) + self.b)

### Model 1 - Baseline

In [40]:
inputs = keras.Input(X_train.shape[1])

x = SimpleLayer(256, activation='relu')(inputs)

outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

2024-01-05 20:16:48.423108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-05 20:16:48.443041: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-05 20:16:48.443234: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [41]:
metrics = [keras.metrics.BinaryAccuracy(),
                      keras.metrics.Precision(),
                      keras.metrics.Recall()]

In [42]:
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.BinaryCrossentropy(),
             metrics=metrics)

In [43]:
history = model.fit(X_train, y_train, epochs=10, validation_split=.2)

Epoch 1/10


2024-01-05 20:17:01.214767: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-05 20:17:01.566133: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f80601d4900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-05 20:17:01.566156: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-01-05 20:17:01.571499: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-05 20:17:01.585741: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1704496621.636189    6748 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2058/2058 [==============================] - 7s 3ms/step - loss: 0.4785 - binary_accuracy: 0.9162 - precision: 0.9036 - recall: 0.8800 - val_loss: 0.0228 - val_binary_accuracy: 0.9999 - val_precision: 1.0000 - val_recall: 0.9997
Epoch 2/10
2058/2058 [==============================] - 5s 2ms/step - loss: 0.0580 - binary_accuracy: 0.9884 - precision: 0.9857 - recall: 0.9847 - val_loss: 0.0044 - val_binary_accuracy: 0.9999 - val_precision: 1.0000 - val_recall: 0.9998
Epoch 3/10
2058/2058 [==============================] - 5s 2ms/step - loss: 0.0476 - binary_accuracy: 0.9945 - precision: 0.9931 - recall: 0.9929 - val_loss: 0.0061 - val_binary_accuracy: 0.9998 - val_precision: 0.9994 - val_recall: 1.0000
Epoch 4/10
2058/2058 [==============================] - 5s 2ms/step - loss: 0.0784 - binary_accuracy: 0.9931 - precision: 0.9914 - recall: 0.9910 - val_loss: 0.0014 - val_binary_accuracy: 0.9998 - val_precision: 0.9998 - val_recall: 0.9997
Epoch 5/10
2058/2058 [=============================